In [ ]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))



!pip install kaggle  # Installed Kaggle API
from google.colab import files

# Uploaded Kaggle API key

# Created a directory for Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d emmarex/plantdisease
!unzip plantdisease.zip -d PlantVillage



import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set dataset path
dataset_path = "PlantVillage"

# Image size for the model
IMG_SIZE = 224
BATCH_SIZE = 32

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values
    rotation_range=30,    # Rotate images randomly
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split into training and validation
)

# Load images from directory
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

print("Class labels:", train_data.class_indices)


TensorFlow Version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open plantdisease.zip, p

FileNotFoundError: [Errno 2] No such file or directory: 'PlantVillage'

In [ ]:
from google.colab import files
files.upload()  # Re-upload the `kaggle.json` file manually


TypeError: 'NoneType' object is not subscriptable

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease
!unzip plantdisease.zip -d PlantVillage


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define parameters
IMG_SIZE = 224  # Resize all images to 224x224
BATCH_SIZE = 32  # Number of images processed at once
DATASET_PATH = "PlantVillage"  # Path to dataset folder

# Data Augmentation and Normalization
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values (0-1)
    rotation_range=30,  # Rotate images randomly
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

# Load Training Data
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Print class labels
print("Class labels:", train_data.class_indices)
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images
images, labels = next(train_data)

# Display 6 sample images
plt.figure(figsize=(10, 6))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(images[i])
    plt.axis("off")
plt.show()





import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load EfficientNetB0 without the top classification layer
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model (so we don't train its weights initially)
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensions
x = Dense(256, activation="relu")(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Prevent overfitting

# Ensure num_classes is defined before using it
num_classes = len(train_data.class_indices) if 'train_data' in globals() else 38
#output_layer = Dense(num_classes, activation="softmax")  # Output layer # This line was defining the layer but not connecting it

# Connect the output layer to the previous layer
output_layer = Dense(num_classes, activation="softmax")(x) # Now, it's connected to the 'x' output of the previous layer


# Define the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()

# Unfreeze the last few layers of EfficientNet
base_model.trainable = True  # Unfreeze the entire model (Optional: Unfreeze only specific layers)

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train again for fine-tuning
EPOCHS_FINE_TUNE = 5  # Adjust based on results
history_fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    verbose=1
)
# Evaluate model after fine-tuning
loss, acc = model.evaluate(val_data)
print(f"Fine-Tuned Model Accuracy: {acc:.4f}")




In [ ]:
model.save("plant_disease_model.keras")  # Recommended Keras format


In [ ]:
from flask import Flask, request, render_template
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model("plant_disease_model.keras")

# Define class labels (same as train_data.class_indices)
class_labels = {0: "Apple Scab", 1: "Apple Black Rot", 2: "Healthy Leaf",  }  # Add all classes

# Function to preprocess the uploaded image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Reshape for model
    return img_array

# Flask Routes
@app.route("/", methods=["GET", "POST"])
def upload_predict():
    if request.method == "POST":
        img_file = request.files["file"]
        if img_file:
            img_path = "static/" + img_file.filename
            img_file.save(img_path)

            # Preprocess and predict
            img_array = preprocess_image(img_path)
            predictions = model.predict(img_array)
            predicted_class = np.argmax(predictions)
            result = class_labels[predicted_class]

            return render_template("index.html", prediction=result, img_path=img_path)

    return render_template("index.html", prediction=None)

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
!pip install fastapi uvicorn


In [ ]:
!pip install fastapi uvicorn


In [ ]:
!pip install pyngrok


In [ ]:

!pip install python-multipart

In [ ]:
import tensorflow as tf
import numpy as np
from io import BytesIO
from PIL import Image
from fastapi import FastAPI, File, UploadFile
import nest_asyncio
import uvicorn

app = FastAPI()

# Load trained model (without optimizer issues)
model = tf.keras.models.load_model("plant_disease_model.keras", compile=False)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Class labels
class_labels = {0: "Apple Scab", 1: "Apple Black Rot", 2: "Healthy Leaf"}

# Image preprocessing function
def preprocess_image(img):
    img = img.resize((224, 224))  # Resize
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Reshape for model
    return img_array

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image_data = await file.read()
    img = Image.open(BytesIO(image_data)).convert("RGB")

    # Preprocess image
    img_array = preprocess_image(img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = float(np.max(predictions))  # Confidence score

    return {"prediction": class_labels[predicted_class], "confidence": confidence}

# Fix async loop issue (for Colab & Jupyter)
nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
